In [1]:
!pip install psycopg2
!pip install pprint

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [382]:
import os
import logging
import numpy as np
import psycopg2
import psycopg2.extensions
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Float, MetaData, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

Подключаемся к БД и выводим результаты запросов для проверки работы базы

In [13]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [24]:
logger.info("Postgres")
params = {
    'host': '0.0.0.0',
    'port': '5433',
    'user': 'postgres'
}
conn = psycopg2.connect(**params)

2018-08-23 21:24:16,316 : INFO : Postgres


In [25]:
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

In [31]:
sql_str = ("""select all_desktop.docuuid, all_desktop.desktop_all_v, content_info.timereadexpected,
(all_desktop.desktop_all_v/all_desktop.desktop_all_s3) as good_readers
from all_desktop inner join content_info on content_info.docuuid = all_desktop.docuuid
where all_desktop.desktop_all_s3<>0 and all_desktop.visits > 3000
group by all_desktop.docuuid, all_desktop.desktop_all_v, content_info.timereadexpected, all_desktop.desktop_all_s3
order by good_readers asc limit 10;""")

In [32]:
cursor.execute(sql_str)
conn.commit()

In [33]:
cursor.fetchall()

[('2638cfae-9c74-11e8-96ad-a25db12b5a53', 14691, 86, 1),
 ('82386dd2-9ba2-11e8-86e9-a25db12b5a53', 4464, 24, 1),
 ('4b3cddae-9a2b-11e8-a40c-1e0e02e95e35', 9954, 45, 1),
 ('42321806-94b7-11e8-9e55-fc60c095174f', 3113, 68, 1),
 ('3d66192a-9490-11e8-9907-2078d824af27', 4913, 48, 1),
 ('6d7eb81e-9af9-11e8-88d9-f017b12b5a53', 1888, 66, 1),
 ('6d99ea58-a125-11e8-85de-8f4a631f500f', 17970, 65, 1),
 ('639f36c2-00ec-11e8-86c6-d6fd8c1371f3', 3334, 257, 1),
 ('108aa054-96fe-11e8-bcbd-2078d824af27', 18668, 62, 1),
 ('9f71df54-a215-11e8-92a5-8f4a631f500f', 6537, 91, 1)]

In [37]:
sql_str_time =("""select docuuid, timereadexpected, desktop_all_v, desktop_all_s3,
max(desktop_all_v/desktop_all_s3)
over (partition by docuuid) as min_scroll
from(select all_desktop.docuuid, content_info.timereadexpected, all_desktop.desktop_all_v, all_desktop.desktop_all_s3 from all_desktop
join content_info on all_desktop.docuuid = content_info.docuuid
where  all_desktop.desktop_all_r > 1000) as sample
order by min_scroll desc limit 10;""")

In [38]:
cursor.execute(sql_str_time)
conn.commit()

In [39]:
cursor.fetchall()

[('78c3ab34-a487-11e8-9645-8f4a631f500f', 106, 13908, 4118, 3),
 ('78c3ab34-a487-11e8-9645-8f4a631f500f', 106, 13908, 4118, 3),
 ('78c3ab34-a487-11e8-9645-8f4a631f500f', 106, 33578, 8957, 3),
 ('78c3ab34-a487-11e8-9645-8f4a631f500f', 106, 33578, 8957, 3),
 ('0814e520-9d3d-11e8-89aa-a25db12b5a53', 351, 61663, 22363, 2),
 ('0814e520-9d3d-11e8-89aa-a25db12b5a53', 357, 61663, 22363, 2),
 ('0814e520-9d3d-11e8-89aa-a25db12b5a53', 355, 61663, 22363, 2),
 ('0814e520-9d3d-11e8-89aa-a25db12b5a53', 351, 61663, 22363, 2),
 ('0814e520-9d3d-11e8-89aa-a25db12b5a53', 348, 61663, 22363, 2),
 ('0814e520-9d3d-11e8-89aa-a25db12b5a53', 351, 61663, 22363, 2)]

Напишем запросы с помощью Pandas.
Для начала подготовим данные

In [578]:
data_content_info = pd.read_csv('id_info.csv', dtype={'visitors': int, 'timeReadExpected': int, 'isLong': int})
data_social_all = pd.read_csv('sharings.csv', dtype={'visitors': int, 'MAX(shareTotal)': int, 'visits_social': int, 'social_R': int, 'social_V': int, 'social_S3': int})
data_all_desktop = pd.read_csv('all_desktop.csv', dtype={'visitors': int, 'desktop_all_R': int, 'desktop_all_V': int, 'desktop_all_S3': int})
data_search_all = pd.read_csv('searching.csv', dtype={'visitors': int, 'desktop_search_R': int,'desktop_search_V': int, 'desktop_search_V': int})

Первый запрос: какие материалы пользователи читали лучше всего. 
Выводим с фильтром по количеству читателей, сортируем по отношению трафика к трафику по доскроллу

In [420]:
ds_all = data_all_desktop.merge(data_content_info, 'left', on = 'docUuid').drop(['visitors_x','visitors_y','date','isLong'], axis = 1)

In [421]:
ds_all.head(2) #проверим как сработало

,docUuid,desktop_all_R,desktop_all_V,desktop_all_S3,timeReadExpected
0,d138d0c6-9bf6-11e8-8f8e-eb1702e95e35,313531,434775,323984,68
1,d138d0c6-9bf6-11e8-8f8e-eb1702e95e35,313531,434775,323984,59


In [422]:
ds_all = ds_all[(ds_all['desktop_all_R'] > 100) & (ds_all['desktop_all_S3'] != 0)]

In [423]:
ds_all.insert(3, 'min_scroll', ((ds_all['desktop_all_V']/ds_all['desktop_all_S3'])))

In [424]:
ds_best_readers = ds_all.sort_values(by='min_scroll', ascending=True).drop(['desktop_all_R','desktop_all_V','desktop_all_S3'], axis = 1).groupby(['docUuid','min_scroll']).mean()

In [425]:
ds_best_readers.head()

timeReadExpected
docUuid                              min_scroll                  
040fa184-90b5-11e8-ade0-fc60c095174f 1.156522           41.000000
                                     1.180473           41.000000
04967534-0bdf-11e8-ae62-ea2c588b2a65 1.083682           82.000000
07b77414-9f98-11e8-b572-eb1702e95e35 1.177083           88.571429
                                     1.179144           88.571429

Выводим топ-5 материалов, где лучший доскролл.

In [570]:
data_all_desktop.head(2) #используем данный сет

,docUuid,visitors,desktop_all_R,desktop_all_V,desktop_all_S3
0,d138d0c6-9bf6-11e8-8f8e-eb1702e95e35,525792,313531,434775,323984
1,99301356-9953-11e8-8039-2078d824af27,255872,135563,223889,171358


In [250]:
data_all_desktop_filter = data_all_desktop[(data_all_desktop['desktop_all_S3'] != 0) & (data_all_desktop['visitors'] >5000)]

In [252]:
data_all_desktop_filter.insert(5, 'good_readers', (data_all_desktop_filter['desktop_all_V']/data_all_desktop_filter['desktop_all_S3']))

In [254]:
data_good_scroll = data_all_desktop_filter.sort_values(by='good_readers', ascending=True)

In [255]:
data_good_scroll.head()

,docUuid,visitors,desktop_all_R,desktop_all_V,desktop_all_S3,good_readers
212,a5dc0066-a154-11e8-84d8-8f4a631f500f,12031,5509,6382,6094,1.047260
197,a5dc0066-a154-11e8-84d8-8f4a631f500f,13494,8536,9770,9307,1.049748
307,1a789cb2-9b07-11e8-b29c-f017b12b5a53,5520,1885,2356,2240,1.051786
276,1a789cb2-9b07-11e8-b29c-f017b12b5a53,7038,2741,3472,3301,1.051802
304,1a789cb2-9b07-11e8-b29c-f017b12b5a53,5593,3216,3998,3800,1.052105


Выводим топ-5 материалов по посетителям за определенный день

In [338]:
top5_for_day = data_content_info[(data_content_info['date'] == '2018-08-21')].sort_values(by='visitors', ascending=False)

In [339]:
top5_for_day.head()

,docUuid,visitors,date,isLong,timeReadExpected
25,50d199a0-a460-11e8-98a1-41d95f45639c,111587,2018-08-21,0,26
33,3a0c5bde-a52c-11e8-bd20-41d95f45639c,99190,2018-08-21,0,81
79,f4d79d92-a519-11e8-a628-8f4a631f500f,40477,2018-08-21,0,45
85,78c3ab34-a487-11e8-9645-8f4a631f500f,38477,2018-08-21,1,106
86,bc306efe-a44e-11e8-b3aa-41d95f45639c,36515,2018-08-21,0,58


Выведем минимальное количество пользователей, которые стали читателями на десктопной версии сайта для материалов, на которых более 1000 посетителей

In [581]:
data_all_desktop.head(2) #используем данный сет

,docUuid,visitors,desktop_all_R,desktop_all_V,desktop_all_S3
0,d138d0c6-9bf6-11e8-8f8e-eb1702e95e35,525792,313531,434775,323984
1,99301356-9953-11e8-8039-2078d824af27,255872,135563,223889,171358


In [588]:
data_all_min = data_all_desktop[(data_all_desktop['visitors'] > 1000)].sort_values(by='desktop_all_R', ascending = True)

In [594]:
data_all_min.groupby(by='docUuid').mean().sort_values(by='desktop_all_R', ascending=True).head(5)

,visitors,desktop_all_R,desktop_all_V,desktop_all_S3
docUuid,,,,
d95fc914-170b-11e8-b027-36686c0143b9,2846.666667,218.333333,406.0,180.333333
e4525d10-6962-11e8-bf55-fb1b3249e00a,1011.000000,264.000000,770.0,687.000000
9258c4e2-9188-11e8-baef-fc60c095174f,1734.333333,549.333333,1365.0,582.333333
d1983f34-7072-11e8-9b2e-ab7403c1ce92,1096.000000,578.000000,974.0,621.000000
f8f22494-91ad-11e8-ac8f-fc60c095174f,1520.000000,623.000000,887.0,655.000000


Смотрим количество посетителей на длинных статьях (islong = 1) за два дня - 20 и 21 августа

In [521]:
data_content_info.head(2)

,docUuid,visitors,date,isLong,timeReadExpected
0,d138d0c6-9bf6-11e8-8f8e-eb1702e95e35,525792,2018-08-10,0,68
1,99301356-9953-11e8-8039-2078d824af27,255872,2018-08-06,0,35


In [522]:
top_for_2days = data_content_info[(data_content_info['date'] > '2018-08-19') & (data_content_info['isLong'] == 1)]

In [523]:
top_for_2days

,docUuid,visitors,date,isLong,timeReadExpected
85,78c3ab34-a487-11e8-9645-8f4a631f500f,38477,2018-08-21,1,106
145,78c3ab34-a487-11e8-9645-8f4a631f500f,19143,2018-08-20,1,106
187,69f972a4-a168-11e8-9077-41d95f45639c,14266,2018-08-20,1,165
312,c43b5286-a146-11e8-bf2c-8f4a631f500f,5363,2018-08-20,1,134
416,69f972a4-a168-11e8-9077-41d95f45639c,2271,2018-08-21,1,165
465,0814e520-9d3d-11e8-89aa-a25db12b5a53,1554,2018-08-21,1,351
502,96de7c24-00eb-11e8-8743-a08db8bcf5f7,1176,2018-08-20,1,289
612,96de8e12-00eb-11e8-8743-a08db8bcf5f7,711,2018-08-21,1,277
657,c43b5286-a146-11e8-bf2c-8f4a631f500f,601,2018-08-21,1,126
668,d95fc914-170b-11e8-b027-36686c0143b9,586,2018-08-20,1,502


In [369]:
top_for_2days.insert(2, 'counter_visit', top_for_2days['docUuid'].count()) #вот тут у меня не получилось корректно посчитать

In [569]:
top_for_2days.head(2)

,docUuid,visitors,date,isLong,timeReadExpected
85,78c3ab34-a487-11e8-9645-8f4a631f500f,38477,2018-08-21,1,106
145,78c3ab34-a487-11e8-9645-8f4a631f500f,19143,2018-08-20,1,106


Смотрим разницу среднего поискового трафика и поискового трафика за каждый день

In [557]:
data_search_all.head(2) #используем этот сет

,docUuid,visitors,desktop_search_R,desktop_search_V,desktop_search_S3
0,d138d0c6-9bf6-11e8-8f8e-eb1702e95e35,525792,1618,2040,1673
1,99301356-9953-11e8-8039-2078d824af27,255872,46,76,53


In [558]:
traffic_dif_filter = data_search_all[(data_search_all['visitors'] >= 1000)].drop(['desktop_search_R','desktop_search_S3','visitors'], axis = 1)

In [560]:
groups = traffic_dif_filter.groupby(by='docUuid')

In [561]:
traffic_dif_filter['descktop_search_V_mean_by_id'] = groups['desktop_search_V'].transform('mean')

In [562]:
traffic_dif_filter.insert(2, 'traffic_dif', traffic_dif_filter['desktop_search_V'] - traffic_dif_filter['descktop_search_V_mean_by_id'])

In [567]:
traffic_dif_filter.sort_values(by='traffic_dif', ascending = False).head(5)

,docUuid,desktop_search_V,traffic_dif,descktop_search_V_mean_by_id
5,ae8e992c-a08b-11e8-9082-41d95f45639c,2162,1679.714286,482.285714
0,d138d0c6-9bf6-11e8-8f8e-eb1702e95e35,2040,1627.125000,412.875000
26,510a1d42-9c87-11e8-9dc5-a25db12b5a53,2317,1553.909091,763.090909
8,3d66192a-9490-11e8-9907-2078d824af27,1826,1515.777778,310.222222
19,1a789cb2-9b07-11e8-b29c-f017b12b5a53,1906,1503.666667,402.333333


Сохраним в CSV

In [568]:
traffic_dif_filter.sort_values(by='traffic_dif', ascending = False).to_csv('search_diviation.csv')